---
title: "Project: CAESB - getting lat lon from address"
format: html
---


### Pacotes

Carregando pacotes que serão usados na transformação da tabela e na busca da latitude e longitude


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from sqlalchemy import create_engine

### Carregando o dataframe com os dados da CAESB


In [ ]:
df = pd.read_csv('data.csv')

#### Separando a segunda coluna em novas linhas

*Transformação 1*


In [ ]:
(
    df.set_index(['Unnamed: 0']) 
       .apply(lambda col: col.str.split(','))
       .explode(['Áreas AfetadasFilter by Áreas Afetadas'])
       .reset_index()
       .reindex(df.columns, axis=1)
       
)

df = df.set_axis(["id", "ra", "end", "inicio", "fim", "tipo", "motivo"], axis=1)

In [ ]:
df["end_completo"] =  df["end"] + ", " + df['ra'] + ", DF" + ", BRAZIL"
df

### Aplicando a função pegar endereço e retornar latlong

*Transformação 2* criando o endereço e procurando a latitude e longitude


In [ ]:
loc = Nominatim(user_agent="mygeo")
geocode = RateLimiter(loc.geocode, min_delay_seconds=1)

df["end_completo"] = df["end"] + ", " + df['ra'] + ", DF" + ", BRAZIL"

df["geo"] = df["end_completo"].apply(geocode)

df

In [ ]:
df["point"] = df["geo"].apply(lambda l: tuple(l.point) if l else None)
df["lat"] = df["geo"].apply(lambda l: l.latitude if l else None)
df["lon"] = df["geo"].apply(lambda l: l.longitude if l else None)

df

**backup in CSV**

In [ ]:
# base com lat lon
df.to_csv("data_geo.csv")

In [ ]:
# base sem lat lon
df_na = df.query("point != point")

df_na

In [ ]:
#df_na.to_csv("data_na.csv")

In [ ]:
#engine = create_engine('sqlite:///caesb.db', echo=False)

#df.to_sql('base', con=engine, if_exists='append')